In [22]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  6 17:13:02 2021

@author: wyzzh
"""
import os
from skimage import io
from skimage.morphology import disk, erosion, dilation, white_tophat, reconstruction
from skimage.measure import label, regionprops, regionprops_table
import numpy as np
from PIL import Image
import tifffile as tif
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tophat_disk_size = 10
bg_thres = 20
erode_size = 2
cut_margin = True 
margin_size = 5

imgFile = r"D:\Work\DLA_test\samples\X0Y0R1W1C0_488.tif"
img = io.imread(imgFile) # Read image

In [40]:
if len(img.shape)==3: # Determine if the image is a stack file with multiple slices
    img = np.mean(img, axis=0) # If true, average the image
else:
    pass # If already averaged, go on processing

tophat_disk = disk(tophat_disk_size) # create tophat structural element disk, diam = tophat_disk_size (typically set to 10)
tophat_img = white_tophat(img, tophat_disk) # Filter image with tophat
top_img_subbg = tophat_img 
top_img_subbg[tophat_img<bg_thres]=0 # clean out array elements smaller than bg_thres, usually set to 40
binary_img = top_img_subbg 
binary_img[binary_img>0]=1 # binarise the image, non-positive elements will be set as 1
erode_disk = disk(erode_size) # create erode structural element disk, diam = erode_size (typically set to 2)
erode_img = erosion(top_img_subbg, erode_disk) # erode image, remove small dots (possibly noise)
dilate_img = dilation(erode_img, erode_disk) # dilate image, recover eroded elements

if cut_margin is True:
    margin = np.ones(np.shape(img))
    margin[0:margin_size, :] = 0
    margin[-margin_size:, :] = 0
    margin[:, 0:margin_size] = 0
    margin[:, -margin_size:] = 0
    mask = dilate_img*margin
else:
    mask = dilate_img
masked_img = mask*img # return masked image

inverse_mask = 1-mask
img_bgonly = inverse_mask*img
seed_img = np.copy(img_bgonly) #https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_holes_and_peaks.html
seed_img[1:-1, 1:-1] = img_bgonly.max()
seed_mask = img_bgonly
filled_img = reconstruction(seed_img, seed_mask, method='erosion')
img_nobg = abs(img - filled_img)

# Label the image to index all aggregates
labeled_img = label(mask)

In [47]:
io.imsave(r'D:\Work\DLA_test\X0Y0R1W1C0_488_x.tif', masked_img)

In [43]:
intensity_list = []
Abs_frame = []
Channel = []
Slice = []
Frame = []

# Get the number of particles
num_aggregates = int(np.max(labeled_img))
# Get profiles of labeled image
df = regionprops_table(labeled_img, intensity_image=img, properties=['label', 'area', 'centroid', 'bbox'])
df = pd.DataFrame(df)
df.columns = ['', 'NAera', 'X_(px)', 'Y_(px)', 'xMin', 'yMin', 'xMax', 'yMax']
# Analyze each particle
for j in range(0, num_aggregates):
    current_aggregate = np.copy(labeled_img)
    current_aggregate[current_aggregate != j + 1] = 0
    current_aggregate[current_aggregate > 0] = 1
    intensity = np.sum(current_aggregate * img_nobg)
    intensity_list.append(intensity)

    Abs_frame.append(1)
    Channel.append(1)
    Slice.append(1)
    Frame.append(1)

df['Abs_frame'] = Abs_frame
df['Channel']= Channel
df['Slice'] = Slice
df['Frame'] = Frame
df['IntegratedInt'] = intensity_list

In [44]:
df

,,NAera,X_(px),Y_(px),xMin,yMin,xMax,yMax,Abs_frame,Channel,Slice,Frame,IntegratedInt
0,1,5,5.200000,34.600000,5,33,7,37,1,1,1,1,76.18
1,2,18,13.000000,82.500000,11,80,16,86,1,1,1,1,476.36
2,3,21,13.000000,175.000000,11,172,16,179,1,1,1,1,502.32
3,4,139,18.172662,72.374101,12,65,25,81,1,1,1,1,8341.38
4,5,13,21.000000,453.000000,19,451,24,456,1,1,1,1,425.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,134,33,480.818182,212.727273,478,209,485,217,1,1,1,1,1065.18
134,135,51,488.745098,262.431373,485,259,494,268,1,1,1,1,2395.23
135,136,32,489.718750,153.250000,487,150,493,158,1,1,1,1,1199.79
136,137,56,502.142857,260.071429,498,256,507,266,1,1,1,1,3354.76
